In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'movies_data3.csv')
df.sample(5)

,imdb_title_id,original_title,year,genre,director,writer,description,actors,avg_vote,votes,metascore1,full_cast
71510,tt3504158,chung fung jin ging,2013,Action Drama,dennislaw,dennislaw,dedicated police officer take pride duty looki...,simonyam suetlam samlee mengmengniu,0.48,0.000075,0.240038,dennislaw simonyam suetlam samlee mengmengniu
9782,tt0052327,the two-headed spy,1958,Drama Thriller War,andrédetoth,j.alvinkugelmass,based upon true story p scotland autobiography...,jackhawkins giascala erikschumann alexanderknox,0.70,0.000262,0.350131,andrédetoth j.alvinkugelmass jackhawkins giasc...
29160,tt0113971,nic smiesznego,1995,Comedy Drama,marekkoterski,marekkoterski,track hilariously bumbling calamity ridden lif...,cezarypazura marekkondrat ewablaszczyk henrykb...,0.74,0.001299,0.370650,marekkoterski cezarypazura marekkondrat ewabla...
22373,tt0090565,4 aventures de reinette et mirabelle,1987,Comedy Drama Romance,éricrohmer,éricrohmer,reinette mirabelle two young girl reinette lif...,joëllemiquel jessicaforde mr.housseau mme.hous...,0.77,0.000827,0.385414,éricrohmer joëllemiquel jessicaforde mr.housse...
41852,tt0317919,mission: impossible iii,2006,Action Adventure Thriller,j.j.abrams,alexkurtzman,imf agent ethan hunt come conflict dangerous s...,tomcruise philipseymourhoffman vingrhames bill...,0.69,0.142313,0.416157,j.j.abrams alexkurtzman tomcruise philipseymou...


In [3]:
df.shape

(85854, 12)

In [4]:
from pywebio.output import put_table,put_image,put_link,put_markdown,put_text
from pywebio.input import input,TEXT,NUMBER,FLOAT,select,input_group

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from youtubesearchpython import VideosSearch
import json, requests

api_key = 'ae425b92085fb0baed654d771acaed36'

In [6]:
def metascore(mylist,df,vect):
    name=[]
    vectorizer = CountVectorizer(lowercase = False)
    for i in mylist:
        if i == 'genre':
            vectorizer = CountVectorizer(lowercase = False,ngram_range=(1,3))
        data_vect = vectorizer.fit_transform(df[i].fillna('None'))
        vect1 = vectorizer.transform(vect[i])
        sim_score = cosine_similarity(data_vect,vect1)
        name.append('sim_score_'+i)
        df['sim_score_'+i] = sim_score
    return name, df

In [7]:
def string_match(name, th = 0.8):
    name = np.array(name)
    vectorizer = CountVectorizer(lowercase = False)
    i = 0
    while(1):
        s1 = name[[i]]
        s2 = name[i+1:]
        try:
            vec1 = vectorizer.fit_transform(s1)
        except:
            i+=1
            continue
        vec2 = vectorizer.transform(s2)
        sim_score = list(enumerate(cosine_similarity(vec1,vec2).ravel(),start=i+1))
        idx = []
        for j in sim_score:
            if j[1]>th:
                idx.append(j[0])
        if idx:
            name = np.delete(name,idx)
            
        i+=1
        if (len(name)-1)<=i:
            break
            
    return name.tolist()

In [8]:
def movie_posters(names):
    posters = {}
    for i in names:
        s = i.replace(' ','%20')
        url = 'https://api.themoviedb.org/3/search/'+'movie'+'?api_key='+api_key+'&language=en-US&query=%27'+s+'%27&page=1&include_adult=false'
        response = requests.get(url)
        val = json.loads(response.text)
        try:
            posters[val['results'][0]['title'] + '(' + val['results'][0]['release_date'][:4] + ')'] = 'https://image.tmdb.org/t/p/w185/' + val['results'][0]['poster_path']
        except:
            pass
        if len(posters) == 10:
            break
        
    return posters

In [9]:
def trending_movies():
    url='https://api.themoviedb.org/3/movie/popular?api_key='+api_key+'&language=en-US&page=1'
    response = requests.get(url)
    val = json.loads(response.text)
    
    trending = {}
    for i in range(len(val['results'])):
        trending[val['results'][i]['original_title'] + '(' + val['results'][i]['release_date'][:4] + ')'] = 'https://image.tmdb.org/t/p/w185/' + val['results'][i]['poster_path']
        if i == 9:
            break
    return trending

In [10]:
def tmdb_recommendation(idx,s,e):
    tmdb = []
    
    for j in range(5):
        url='https://api.themoviedb.org/3/movie/'+str(idx)+'/recommendations?api_key='+api_key+'&language=en-US&page='+str(j+1)
        response = requests.get(url)
        val = json.loads(response.text)
        if not val['results']:
            break
        for i in range(len(val['results'])):
            if (int(val['results'][i]['release_date'][:4]) >= s) and (int(val['results'][i]['release_date'][:4]) <= e):
                tmdb.append(val['results'][i]['original_title'])
            if len(tmdb) == 20:
                return tmdb           

In [11]:
def get_cast_dict(df,name,n):
    d = {}
    for i in range(n):
        if df['profile_path'].values[i]:
            profile_path = 'https://image.tmdb.org/t/p/w185/' + df['profile_path'].values[i]
        else:
            profile_path = 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'
        d[df[name].values[i]] = profile_path
        if i == (df.shape[0]-1):
            break 
    return d

In [12]:
def movies_cast(idx):
    url='https://api.themoviedb.org/3/movie/'+str(idx)+'/credits?api_key='+api_key+'&language=en-US'
    response = requests.get(url)
    val = json.loads(response.text) 
    gender = {1:'Female',2:'Male'}
    
    a = {}
    idx = []
    cast = pd.DataFrame(val['cast'])
    cast['character_name'] = cast['original_name'] + ' as ' + cast['character']
    cast = cast[cast['known_for_department']=='Acting']
    cast_dict = get_cast_dict(cast,'character_name',8)
    if cast['profile_path'].values[0]:
        profile_path = 'https://image.tmdb.org/t/p/w185/' + cast['profile_path'].values[0]
    else:
        profile_path = 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'
    a['profile_path'] = profile_path
    a['name'] = ['Original_name : ' + '\'' + cast['original_name'].values[0] + '\'']
    a['gender'] = ['Gender : ' + gender[cast['gender'].values[0]]]
    a['department'] = ['Department : ' + cast['known_for_department'].values[0]]
    a['pop'] = ['Popularity : ' + str(cast['popularity'].values[0])]
    idx.append(cast['id'].values[0])
    
    d = {}
    cast = pd.DataFrame(val['crew'])
    directors = cast[cast['department']=='Directing']
    directors_dict = get_cast_dict(directors,'original_name',4)
    if directors['profile_path'].values[0]:
        profile_path = 'https://image.tmdb.org/t/p/w185/' + directors['profile_path'].values[0]
    else:
        profile_path = 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'
    d['profile_path'] = profile_path
    d['name'] = ['Original_name : ' + '\'' + directors['original_name'].values[0] + '\'']
    d['gender'] = ['Gender : ' + gender[directors['gender'].values[0]]]
    d['department'] = ['Department : ' + directors['known_for_department'].values[0]]
    d['pop'] = ['Popularity : ' + str(directors['popularity'].values[0])]
    idx.append(directors['id'].values[0])
    
    w = {}
    writers = cast[cast['department']=='Writing']
    writers_dict = get_cast_dict(writers,'original_name',4)
    if writers['profile_path'].values[0]:
        profile_path = 'https://image.tmdb.org/t/p/w185/' + writers['profile_path'].values[0]
    else:
        profile_path = 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'
    w['profile_path'] = profile_path
    w['name'] = ['Original_name : ' + '\'' + writers['original_name'].values[0] + '\'']
    w['gender'] = ['Gender : ' + gender[writers['gender'].values[0]]]
    w['department'] = ['Department : ' + writers['known_for_department'].values[0]]
    w['pop'] = ['Popularity : ' + str(writers['popularity'].values[0])]
    idx.append(writers['id'].values[0])
    
    return cast_dict,directors_dict,writers_dict,a,d,w,idx

In [13]:
def top_cast_details(idx,movie_name):
    url = 'https://api.themoviedb.org/3/person/'+str(idx)+'/movie_credits?api_key='+api_key+'&language=en-US'
    response = requests.get(url)
    val = json.loads(response.text)
    temp = pd.DataFrame(val['cast'])
    try:
        temp.sort_values(by = 'popularity',ascending=False,inplace=True,ignore_index=True)
        data = {}
        name = {}

        for i in range(30):
            if (temp['original_title'][i] == movie_name):
                continue
            title = temp['original_title'][i] + '(' + temp['release_date'][i][:4] + ')'
            if temp['poster_path'][i]:
                poster_path = 'https://image.tmdb.org/t/p/w185/' + temp['poster_path'][i]
            else:
                poster_path = 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'
            data[title] = poster_path
            name[temp['original_title'][i]] = title

            if i == (temp.shape[0]-1):
                break

        s = list(name.keys())   
        s = string_match(s, th = 0.8)

        d = {}
        for i in name.keys():
            if i in s:
                d[name[i]] = data[name[i]]
            if len(d) >= 8:
                break

        return d
    except:
        return {}

In [14]:
def similarity(movie, mylist = ['genre','full_cast'], weight='balanced', pop = 5, start_year = 2000, end_year = 2020, min_rating = 5.0, Total_votes = 100000):
    data = {}
    df = pd.read_csv(r'movies_data3.csv') 
    s = movie.replace(' ','%20')
    url = 'https://api.themoviedb.org/3/search/'+'movie'+'?api_key='+api_key+'&language=en-US&query=%27'+s+'%27&page=1&include_adult=false'
    response = requests.get(url)
    val = json.loads(response.text)
    if not val['results']:
        return [],[],[]
    data['id'] = val['results'][0]['id']
    if val['results'][0]['poster_path']:
        data['poster'] = 'https://image.tmdb.org/t/p/w185/' + val['results'][0]['poster_path']
    else:
        data['poster'] = 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'
    data['name'] = ['Name : ' + '\'' + val['results'][0]['title'] + '\'']
    movie_name = val['results'][0]['title']
    data['language'] = ['Language : ' + val['results'][0]['original_language']]
    data['release_date'] = ['Release Date : ' + val['results'][0]['release_date']]
    data['ratings'] = ['Ratings : ' + str(val['results'][0]['vote_average']) + ' (' + str(val['results'][0]['vote_count']) + ')']
    data['overview'] = [val['results'][0]['overview']]
    trailer_search = val['results'][0]['title'] + ' ' + val['results'][0]['release_date'][:4] + ' official' + ' trailer'
    videosSearch = VideosSearch(trailer_search, limit = 1)
    result = videosSearch.result()
    data['trailer_link'] = result['result'][0]['link']
    url = 'https://api.themoviedb.org/3/movie/'+str(data['id'])+'?api_key='+api_key+'&language=en-US'
    response = requests.get(url)
    val2 = json.loads(response.text)
    genres = []
    for i in range(len(val2['genres'])):
        genres.append(val2['genres'][i]['name'])
        
    data['genres'] = ['Genres : ' + ', '.join(genres)]
    data['imdb_id'] = val2['imdb_id']
    data['duration'] = ['Duration : ' + str(val2['runtime']) + ' minutes']
    data['tagline'] = ['Tagline : ' + val2['tagline']]
      
    vect = df.loc[df['imdb_title_id'] == str(data['imdb_id'])]
    if vect.empty:
        name = tmdb_recommendation(data['id'], start_year, end_year)
    else:
        min_rating = (min_rating/10.0)
        Total_votes = (Total_votes/2278845.0)
        df = df[((df.year >= start_year) & (df.year <= end_year)) & (df.votes >= Total_votes) & (df.avg_vote >= min_rating)]
        if df.empty:
            name = []
        else:
            name,df = metascore(mylist,df,vect)
            if (weight == 'balanced') | (len(mylist) == 1):
                weights=[(1.0 / len(mylist))] * len(mylist)
            else:
                s = sum(weight)
                weights = [i/s for i in weight]
            df['metascore2'] = 0.0
            for i in range(len(name)):
                df['metascore2'] = df['metascore2'] + df[name[i]]*weights[i]
            my_weight=(pop / 10.0)
            df['metascore'] = df['metascore1']*my_weight + df['metascore2']*(1.0-my_weight)
            sorted_data = df.sort_values(by = ['metascore'], ascending=False)
            name = sorted_data.iloc[:31,[1]].values.ravel().tolist() 
            if val['results'][0]['title'].lower() in name:
                name.remove(val['results'][0]['title'].lower())
    if name: 
        name = string_match(name,th=0.8)
        name = movie_posters(name)

    return data,name,movie_name

In [15]:
start_date = 1990
end_date = 2021
popularity = 0.3
rating = 5.0
no_votes = 1000
mylist = ['genre']
myweight = 'balanced'
movie_name = 'the green mile'

data,name,movie_name = similarity(movie_name,mylist,myweight,popularity,start_date,end_date,rating,no_votes)

In [16]:
name

{'Let the Right One In(2008)': 'https://image.tmdb.org/t/p/w185//7IG4WjaAOVDlLvLUkh513HSwhW8.jpg',
 'Lilies(1996)': 'https://image.tmdb.org/t/p/w185//uKmFhaPD0EzqJr0tjvXclr11FkA.jpg',
 'Border(2018)': 'https://image.tmdb.org/t/p/w185//8KzOTRlKjdAKQTemV41hq86ldFS.jpg',
 'The Frame(2014)': 'https://image.tmdb.org/t/p/w185//82Tpv6o0zpkN2OsNXYK3feBRe10.jpg',
 'Sicilian Ghost Story(2017)': 'https://image.tmdb.org/t/p/w185//bqKVM6ujKR1UkyehoRaBsxw2D8w.jpg',
 'Needful Things(1993)': 'https://image.tmdb.org/t/p/w185//9RxE6kd1ky4689WxqX6CHEpIWya.jpg',
 'The Invisible Man(2020)': 'https://image.tmdb.org/t/p/w185//5EufsDwXdY2CVttYOk2WtYhgKpa.jpg',
 'The Warning(2018)': 'https://image.tmdb.org/t/p/w185//vpiXfao6p0JonETFaLVhrpIkPTy.jpg',
 'Cosmopolis(2012)': 'https://image.tmdb.org/t/p/w185//94m7J9YeO0W7IEbfsOtFbweGgcu.jpg',
 'Pulp Fiction(1994)': 'https://image.tmdb.org/t/p/w185//d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg'}

In [17]:
data

{'id': 497,
 'poster': 'https://image.tmdb.org/t/p/w185//velWPhVMQeQKcxggNEU8YmIo52R.jpg',
 'name': ["Name : 'The Green Mile'"],
 'language': ['Language : en'],
 'release_date': ['Release Date : 1999-12-10'],
 'ratings': ['Ratings : 8.5 (12463)'],
 'overview': ["A supernatural tale set on death row in a Southern prison, where gentle giant John Coffey possesses the mysterious power to heal people's ailments. When the cell block's head guard, Paul Edgecomb, recognizes Coffey's miraculous gift, he tries desperately to help stave off the condemned man's execution."],
 'trailer_link': 'https://www.youtube.com/watch?v=Ki4haFrqSrw',
 'genres': ['Genres : Fantasy, Drama, Crime'],
 'imdb_id': 'tt0120689',
 'duration': ['Duration : 189 minutes'],
 'tagline': ['Tagline : Miracles do happen.']}

In [18]:
cast_dict, directors_dict, writers_dict, a, d, w, idx = movies_cast(data['id'])

actor = top_cast_details(idx[0],movie_name)
director = top_cast_details(idx[1],movie_name)
writer = top_cast_details(idx[2],movie_name)

In [19]:
a

{'profile_path': 'https://image.tmdb.org/t/p/w185//xndWFsBlClOJFRdhSt4NBwiPq2o.jpg',
 'name': ["Original_name : 'Tom Hanks'"],
 'gender': ['Gender : Male'],
 'department': ['Department : Acting'],
 'pop': ['Popularity : 21.652']}

In [20]:
d

{'profile_path': 'https://image.tmdb.org/t/p/w185//nvOqAQhKtXdHtczgqanoCfltsxJ.jpg',
 'name': ["Original_name : 'Frank Darabont'"],
 'gender': ['Gender : Male'],
 'department': ['Department : Directing'],
 'pop': ['Popularity : 5.082']}

In [21]:
w

{'profile_path': 'https://image.tmdb.org/t/p/w185//7r5nEzNanuEhmxtpsKE1uCBU5Jd.jpg',
 'name': ["Original_name : 'Stephen King'"],
 'gender': ['Gender : Male'],
 'department': ['Department : Writing'],
 'pop': ['Popularity : 3.476']}

In [22]:
actor

{'Toy Story(1995)': 'https://image.tmdb.org/t/p/w185//uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg',
 'The Simpsons Movie(2007)': 'https://image.tmdb.org/t/p/w185//s3b8TZWwmkYc2KoJ5zk77qB6PzY.jpg',
 'Forrest Gump(1994)': 'https://image.tmdb.org/t/p/w185//h5J4W4veyxMXDMjeNxZI46TsHOb.jpg',
 'Cars(2006)': 'https://image.tmdb.org/t/p/w185//qa6HCwP4Z15l3hpsASz3auugEW6.jpg',
 'News of the World(2020)': 'https://image.tmdb.org/t/p/w185//fYQCgVRsQTEfUrP7cW5iAFVYOlh.jpg',
 'Sully(2016)': 'https://image.tmdb.org/t/p/w185//5L29r3uwDe49XDMaK2xDkDht9t0.jpg',
 'Saving Private Ryan(1998)': 'https://image.tmdb.org/t/p/w185//1wY4psJ5NVEhCuOYROwLH2XExM2.jpg',
 'The Polar Express(2004)': 'https://image.tmdb.org/t/p/w185//zCuq1sy3xwbJJytFttp2qItqdc8.jpg'}

In [23]:
director

{'The Making of The Walking Dead(2010)': 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg',
 'Fear Himself: The Life and Crimes of Freddy Krueger(2012)': 'https://image.tmdb.org/t/p/w185//yDhdUPsXoyISfFEY0PQVNQnWaAw.jpg',
 'Drew: The Man Behind the Poster(2013)': 'https://image.tmdb.org/t/p/w185//xZ0lzSIEmwa7yBx2KwtsBKLkhz7.jpg',
 'Stephen King: Shining in the Dark(1999)': 'https://image.tmdb.org/t/p/w185//1F0bQ77FBIEk0PbS3vV8mbLgZSj.jpg',
 'Fantastic Flesh: The Art of Make-Up EFX(2008)': 'https://image.tmdb.org/t/p/w185//mLtA5V91BLALD7Kf4T44fcVWKfP.jpg',
 'Tales from the Script(2009)': 'https://image.tmdb.org/t/p/w185//dnKAvhpt9unt7nBx86Ma4bk9aFc.jpg',
 'Nightmare Factory(2011)': 'https://image.tmdb.org/t/p/w185//aKTceKHI7kCzs6LLFr5262HePMS.jpg',
 'The United Monster Talent Agency(2010)': 'https://www.canadaid.ca/wp-content/uploads/2019/02/no-image-available-180x300.jpg'}

In [24]:
writer

{'It Chapter Two(2019)': 'https://image.tmdb.org/t/p/w185//zfE0R94v1E8cuKAerbskfD3VfUt.jpg',
 'Pet Sematary(1989)': 'https://image.tmdb.org/t/p/w185//a1gIACZb04bL8EvLqMpofW2Eqeo.jpg',
 'Creepshow(1982)': 'https://image.tmdb.org/t/p/w185//4SoyTCEpsgLjX6yAyMsx3AsAyRQ.jpg',
 'Stuck in Love(2013)': 'https://image.tmdb.org/t/p/w185//6vG0eCHT2e4uH035g1s6JTouU6R.jpg',
 'Sleepwalkers(1992)': 'https://image.tmdb.org/t/p/w185//p1mCpwvdui1XfS13Fii1Tn0jC0q.jpg',
 'Diary of the Dead(2008)': 'https://image.tmdb.org/t/p/w185//tGktSJW5dPf8QIjmcqum4KOEZPZ.jpg',
 'Maximum Overdrive(1986)': 'https://image.tmdb.org/t/p/w185//cFrKCumtZMHCWwWNxgGRyaxhYu2.jpg',
 'Thinner(1996)': 'https://image.tmdb.org/t/p/w185//wmnTIiRf3Nl5jhMlqLRXQY5msN6.jpg'}

In [25]:
def print_image(name,path):
    if name:
        if len(name) == 1:
            put_table([[put_image(path[0])]],header = name[:1])

        elif len(name) == 2:
            put_table([[put_image(path[0]),
                        put_image(path[1])]],header = name[:2])

        elif len(name) == 3:
            put_table([[put_image(path[0]),
                        put_image(path[1]),
                        put_image(path[2])]],header = name[:3])

        elif len(name) == 4:
            put_table([[put_image(path[0]),
                        put_image(path[1]),
                        put_image(path[2]),
                        put_image(path[3])]],header = name[:4])
        else:
            put_table([[put_image(path[0]),
                        put_image(path[1]),
                        put_image(path[2]),
                        put_image(path[3]),
                        put_image(path[4])]],header = name[:5])

In [26]:
def pywebio_testing():
    recommend_type = ['genre' ,'genre + original_title', 'genre + full_cast', 'genre + description']
    
    movie = input_group("Movie Recommendation System",[
            input('Enter the name of the movie',placeholder = "(eg: Iron man)", type = TEXT, name = 'movie_name'),
            input('Start year of movies to recommend',placeholder = "(eg: 2000)", name='start_date', type = NUMBER),
            input('End year of movies to recommend',placeholder = "(eg: 2021)", name='end_date', type = NUMBER),
            input('Enter popularity score of movies to recommend',placeholder = 'Number between 0 to 1 (weightage) eg: 0.3', name='pop_score', type = FLOAT),
            input('Enter minimum ratings of movies to recommend',placeholder = 'Number between 0 to 10 (min rating) eg: 5.0', name='min_rating', type = FLOAT),
            input('Enter minimum number of votes for movies to recommend (eg:100000)', name='votes', type = NUMBER),
            select("Select the type of recommendation：",recommend_type,name='type')
            ])
    
    start_date = movie['start_date']
    end_date = movie['end_date']
    popularity = movie['pop_score']
    rating = movie['min_rating']
    no_votes = movie['votes']
    mylist = movie['type'].split(' + ')
    myweight = 'balanced'
    movie_name = movie['movie_name']
    
    data,name,movie_name = similarity(movie_name, mylist, myweight, popularity, start_date, end_date, rating, no_votes)

    if not data:
        put_markdown(r""" ### No movies to recommend check the movie name entered""")
    else:
        temp = []
        for i in ['name','genres','language','release_date','ratings','duration','tagline']:
            temp.append(data[i])

        put_table([[put_table([[put_image(data['poster']),put_table(temp)]],header = ['Movie Poster','Details'])] , 
                   [put_link(name = 'Click here to view trailer',url = data['trailer_link'])],
                   [put_table([data['overview']],header=['Overview'])] ])
        
        cast_dict, directors_dict, writers_dict, a, d, w, idx = movies_cast(data['id'])
        del data
        
        if cast_dict:
            put_markdown(r""" # Top cast from the movie :""")
            names = list(cast_dict.keys())
            path = list(cast_dict.values())
            print_image(names[:4],path[:4])
            print_image(names[4:],path[4:])
            del cast_dict
            
        if directors_dict:
            put_markdown(r""" # Directors of the movie :""")
            names = list(directors_dict.keys())
            path = list(directors_dict.values())
            print_image(names,path)
            del directors_dict
            
        if writers_dict:
            put_markdown(r""" # Writers of the movie :""")
            names = list(writers_dict.keys())
            path = list(writers_dict.values())
            print_image(names,path)
            del writers_dict
        
        actor = top_cast_details(idx[0],movie_name)
        if actor:
            put_markdown(r""" # Top movies of the lead({}) :""".format(a['name'][0].split(' : ')[1]))
            temp = list(a.values())[1:]
            put_table([[put_image(a['profile_path']),put_table(temp)]],header = ['Actor Poster','Details'])
            names = list(actor.keys())
            path = list(actor.values())
            print_image(names[:4],path[:4])
            print_image(names[4:],path[4:])
            del actor
        
        director = top_cast_details(idx[1],movie_name)
        if director:
            if idx[1] == idx[2]:
                put_markdown(r""" # Top movies of the director and writer({}) :""".format(d['name'][0].split(' : ')[1]))
            else:   
                put_markdown(r""" # Top movies of the director({}) :""".format(d['name'][0].split(' : ')[1]))
            temp = list(d.values())[1:]
            put_table([[put_image(d['profile_path']),put_table(temp)]],header = ['Director Poster','Details'])
            names = list(director.keys())
            path = list(director.values())
            print_image(names[:4],path[:4])
            print_image(names[4:],path[4:])
            del director
         
        if idx[1] != idx[2]:
            writer = top_cast_details(idx[2],movie_name) 
            
            if writer:
                put_markdown(r""" # Top movies of the writer({}) :""".format(w['name'][0].split(' : ')[1]))
                temp = list(w.values())[1:]
                put_table([[put_image(w['profile_path']),put_table(temp)]],header = ['Writer Poster','Details'])
                names = list(writer.keys())
                path = list(writer.values())
                print_image(names[:4],path[:4])
                print_image(names[4:],path[4:])
                del writer

        if not name:
            put_markdown(r""" ### No movies to recommend try again with different parameters""")
        else:
            s = 'for movie \'' + movie_name + '\''
            put_markdown(r""" # Recommended Movies({}) :""".format(s))
            names = list(name.keys())
            path = list(name.values())
            print_image(names[:5],path[:5])
            print_image(names[5:],path[5:])
            
        put_markdown(r""" # Trending Movies :""")
        name = trending_movies()
        names = list(name.keys())
        path = list(name.values())
        print_image(names[:5],path[:5])
        print_image(names[5:],path[5:])
        
        put_markdown(r""" ### Created by Nakul Chamariya """)
        put_link(name = 'Github Code Link', url = 'https://github.com/Nakul74/Movie-Recommendation-System')
        put_markdown(r""" #### Contact details : """)
        put_text('Email id : nakulchamariya74@gmail.com')
        put_text('LinkedIn link : www.linkedin.com/in/nakul-chamariya')

In [27]:
pywebio_testing()